In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import torch 
from PIL import Image
from model import Generator, Discriminator

In [2]:
gen = Generator(in_channels=4, out_channels=4, num_upsample_blocks=2, num_residual_blocks=16)
dis = Discriminator(input_channels=4)

In [3]:
img_path = "/Users/archismanchakraborti/Desktop/python_files/Super-Resolution-Tool/data/low_res_data/3_lr.png"
img = Image.open(img_path)


# Convert img to torch tensor 
img = np.array(img)
img_torch = torch.tensor(img).permute(2, 0, 1).unsqueeze(0).float()

In [6]:
print(f"Input image shape = {img_torch.shape}")
out_img = gen(img_torch)
print(f"Generator output shape = {out_img.shape}")
dis_out = dis(out_img)
print(f"Discriminator output shape = {dis_out.shape}, Discriminator output = {dis_out.item()}")

Input image shape = torch.Size([1, 4, 32, 32])
Generator output shape = torch.Size([1, 4, 96, 96])
Discriminator output shape = torch.Size([1, 1]), Discriminator output = 0.5164053440093994
